In [1]:
import runpod
import time
import os
import subprocess
from dotenv import load_dotenv
import pyperclip #pip install pyperclip==1.9.0

In [2]:
load_dotenv(os.path.join(os.path.expanduser("~"), "Desktop", "env"))  # Get desktop path dynamically
runpod.api_key = os.getenv('RUNPOD_API_KEY') #Load API key from environment variable
files_path = os.path.join(os.path.expanduser("~"), "Desktop", "runpod_files") #Path of the files to copy to the pod

Create Pod - Had to modify the files to take 'start_jupyter' as input, as stated in [this](https://github.com/runpod/runpod-python/pull/328/commits/071484c10438f546666667c7a2f38ad143beb435) issue

In [3]:
# Create a pod - https://github.com/runpod/runpod-python/blob/main/runpod/api/ctl_commands.py
pod = runpod.create_pod(name="test", #Set a name for the pod
                        image_name="runpod/pytorch:2.1.0-py3.10-cuda11.8.0-devel-ubuntu22.04", #Set the image to use
                        gpu_type_id="NVIDIA RTX A4500", #runpod.get_gpus() for all gpu types
                        cloud_type="COMMUNITY", #"ALL", "COMMUNITY", "SECURE"
                        support_public_ip=True, #This is the default
                        country_code="FR", #Data should remain within the EU
                        container_disk_in_gb=30, #Set the disk size of the container
                        volume_in_gb=130, #Set the volume size of the pod
                        ports="8888/http,22/tcp", #Set the ports to expose
                        volume_mount_path="/workspace", #Set the volume mount path - Should modified below if changed here
                        start_jupyter=True, #Start the Jupyter notebook - Not exist in official documentation, found through issues.
                        )

In [ ]:
# Get all my pods
pods = runpod.get_pods()
pods #Confirm the pod was created

In [ ]:
time.sleep(90) #Wait for the pod to be ready and get the http url
exposed_pod = runpod.get_pods()[0]
exposed_pod

In [6]:
port_number=int(exposed_pod['ports'].split(',')[1].split('/')[0]) # Extract 22 from '8888/http,22/tcp'

ip, public_port = [port for port in exposed_pod['runtime']['ports'] if port['privatePort'] == port_number][0]['ip'], \
                    [port for port in exposed_pod['runtime']['ports'] if port['privatePort'] == port_number][0]['publicPort']

Connect via SSH to the Pod

In [ ]:
# First connect via SSH to accept the host key
ssh_command = f"ssh -p {public_port} -i ~/.ssh/id_ed25519 -o StrictHostKeyChecking=accept-new root@{ip} exit"
subprocess.run(ssh_command, shell=True, check=True)

Copy files from local PC to the Pod

In [ ]:
source_dir = files_path.replace('\\', '/')
scp_command = f"scp -P {public_port} -i ~/.ssh/id_ed25519 {source_dir}/* root@{ip}:/workspace/"

try:
    result = subprocess.run(scp_command, shell=True, check=True, capture_output=True, text=True)
    print("Successfully copied all files.")
except subprocess.CalledProcessError as e:
    print(f"Error copying files: {e.stderr}")

Create environment and install dependencies inside the Pod

In [ ]:
# SSH command to connect and execute multiple commands - Takes ~3min to complete
ssh_commands = [
    "python -m venv /workspace/myenv",
    "source /workspace/myenv/bin/activate", 
    "cd /workspace",
    "pip install --upgrade ipykernel",
    "python -m ipykernel install --name myenv --user --display-name 'Python (myenv)'",
    "pip install -r requirements.txt && pip install flash-attn==2.6.3" #requirements.txt copied in the previous step
]

# Join commands with semicolons for sequential execution
command_string = "; ".join(ssh_commands)
ssh_command = f"""ssh root@{ip} -p {public_port} -i ~/.ssh/id_ed25519 "{command_string}" """

# Execute the SSH command with live output streaming
process = subprocess.Popen(ssh_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Stream output in real-time
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        print(output.strip())

# Print any errors after completion
if process.returncode != 0:
    print("Error executing commands:")
    print(process.stderr.read())

In [ ]:
# Extract Jupyter password from env variables
jupyter_password = [env.split('=')[1] for env in exposed_pod['env'] if env.startswith('JUPYTER_PASSWORD')][0]
jupyter_port = exposed_pod['ports'].split("/")[0]  # Extract 8888 from "8888/http,22/tcp"
jupyter_url = "https://" + exposed_pod['id'] + "-" + jupyter_port +".proxy.runpod.net/?token=" + jupyter_password
jupyter_url

In [ ]:
#Copy the above variable to the clipboard
pyperclip.copy(jupyter_url)
print("Jupyter URL copied to clipboard!")

Copy files from Pod to local PC

In [ ]:
# # Copy output files from remote to local
# output_files = "*.xlsx,*.png"  # File patterns to copy
# local_dest = os.path.join(os.path.expanduser("~"), "Desktop", "test")

# # Ensure local destination directory exists
# os.makedirs(local_dest, exist_ok=True)

# # Construct scp command to copy files from remote to local
# scp_command = f"scp -P {public_port} -i ~/.ssh/id_ed25519 root@{ip}:/workspace/{{{output_files}}} {local_dest}"

# try:
#     result = subprocess.run(scp_command, shell=True, check=True, capture_output=True, text=True)
#     print(f"Successfully copied output files to {local_dest}")
# except subprocess.CalledProcessError as e:
#     print(f"Error copying output files: {e.stderr}")

Terminate the Pod

In [13]:
# runpod.terminate_pod(pods[0]['id'])
# runpod.get_pods()